# Pandas 2.2.2用

## 0) 前提

* 環境: **Python 3.10.15 / pandas 2.2.2**
* **指定シグネチャ厳守**（関数名・引数名・返却列・順序）
* I/O 禁止、不要な `print` や `sort_values` 禁止（並び替えは `nlargest` を使用）

## 1) 問題

* `Cinema から、ID が奇数かつ description が "boring" ではない映画を抽出し、rating の降順で返す。`
* 入力 DF: `Cinema(id: int, movie: str, description: str, rating: float)`
* 出力: 列は `id, movie, description, rating`。**rating 降順**（同率時の順序は任意）

## 2) 実装（指定シグネチャ厳守）

> 列最小化 → 条件抽出 → `nlargest` で降順（`sort_values` 禁止対応）

```python
import pandas as pd

def select_non_boring_odd_movies(cinema: pd.DataFrame) -> pd.DataFrame:
    """
    Returns:
        pd.DataFrame: 列名と順序は ['id', 'movie', 'description', 'rating']
                      rating の降順で返す（同率時の順序は任意）
    """
    # 必要列のみ抽出（列最小化）
    cols = ['id', 'movie', 'description', 'rating']
    c = cinema.loc[:, cols]

    # 条件: 奇数ID かつ description が 'boring' ではなく（かつ非NULL）
    # 仕様上 NULL を含めないため isna の否定を明示
    mask = (c['id'] % 2 == 1) & c['description'].notna() & (c['description'] != 'boring')
    kept = c.loc[mask]

    # 並び替え: sort_values 禁止のため nlargest を使用して降順を実現
    # n == len(kept) を取れば rating DESC 全件ソートと同等
    out = kept.nlargest(len(kept), columns='rating')

    # 返却: 仕様列・順序のまま
    return out[['id', 'movie', 'description', 'rating']]

Analyze Complexity
Runtime 276 ms
Beats 35.04%
Memory 67.14 MB
Beats 68.84%

```

## 3) アルゴリズム説明

* 使用 API

  * ブールマスク: `Series %`, `Series.notna()`, 比較 `!=`
  * 列最小化: `DataFrame.loc[:, cols]`
  * 降順取得: `DataFrame.nlargest(n, 'rating')`（`sort_values` 非使用要件に対応）
* **NULL / 重複 / 型**

  * `description.notna()` により `NULL` を除外（仕様に合わせて明示）
  * `rating` は数値列必須（float）。文字列混在の場合は事前に `to_numeric` を検討
  * 主キー `id` 前提なので重複行は想定しない

## 4) 計算量（概算）

* フィルタ（ブールマスク）: **O(N)**
* `nlargest(len(kept), 'rating')`: 内部的には選択アルゴリズム＋部分ソートで **O(M log M)**（M は残件数）

  * 全件降順と同等のオーダーだが、`sort_values` を使わず要件を満たす手段として最小限

## 5) 図解（Mermaid 超保守版）

```mermaid
flowchart TD
  A[入力 Cinema DF] --> B[列最小化 id,movie,description,rating]
  B --> C[条件抽出 id 奇数 かつ description 非NULL かつ != 'boring']
  C --> D[nlargest で rating 降順]
  D --> E[出力 id,movie,description,rating]
```

**Pandas のオーバーヘッド削減**と**コピー削減**でまだ詰められます。`sort_values` 禁止の前提は守りつつ、`nlargest` の対象を **Series** にしてインデックスで並べ替える・もしくは **NumPy だけで順位付け**するのが速いです。

---

## 1) 低リスク版（Pandas寄り・最小変更）

ポイント

* 中間の `c = cinema.loc[:, cols]` コピーを削除
* マスクは **NumPy 配列**で作成（`to_numpy()`）
* 並び替えは **Series.nlargest** でインデックスを取得 → 最後に列投影

```python
import pandas as pd
import numpy as np

def select_non_boring_odd_movies(cinema: pd.DataFrame) -> pd.DataFrame:
    """
    Returns:
        pd.DataFrame: ['id', 'movie', 'description', 'rating'] を rating 降順で返す
    """
    id_arr = cinema['id'].to_numpy()
    desc_arr = cinema['description'].to_numpy()
    # NULL 除外 + 'boring' 除外 + 奇数ID
    mask = ((id_arr & 1) == 1) & (desc_arr == desc_arr) & (desc_arr != 'boring')

    # 抽出行の index を得る
    idx = cinema.index[mask]
    # rating の降順 index を Series.nlargest で取得（DataFrame.nlargest より軽い）
    top_idx = cinema.loc[idx, 'rating'].nlargest(idx.size).index

    return cinema.loc[top_idx, ['id', 'movie', 'description', 'rating']]

Analyze Complexity
Runtime 261 ms
Beats 65.91%
Memory 67.39 MB
Beats 28.60%

```

**ねらい**

* `Series.nlargest` は対象列だけを扱うため、`DataFrame.nlargest` よりメモリアクセスが少なく、速くなりやすいです。
* 中間 DataFrame を作らないのでコピー削減（メモリ・CPU ともに軽くなる傾向）。

---

## 2) 攻めの最速版（NumPy 主導）

ポイント

* 並び替えを **`np.argsort`** に完全委譲（Pandas のインデクサ組み立てコストを最小化）
* 全行の `rating` を配列で一度読むだけ

```python
import pandas as pd
import numpy as np

def select_non_boring_odd_movies(cinema: pd.DataFrame) -> pd.DataFrame:
    """
    Returns:
        pd.DataFrame: ['id', 'movie', 'description', 'rating'] を rating 降順で返す
    """
    id_arr   = cinema['id'].to_numpy()
    desc_arr = cinema['description'].to_numpy()
    rate_arr = cinema['rating'].to_numpy()

    mask = ((id_arr & 1) == 1) & (desc_arr == desc_arr) & (desc_arr != 'boring')
    sel = np.flatnonzero(mask)

    # rating 降順の位置（選択部分のみ）を取得
    order_in_sel = np.argsort(rate_arr[sel])[::-1]
    row_pos = sel[order_in_sel]

    return cinema.iloc[row_pos, :][['id', 'movie', 'description', 'rating']]

Analyze Complexity
Runtime 256 ms
Beats 76.61%
Memory 67.11 MB
Beats 68.84%

```

**ねらい**

* `nlargest(len)` は実質「全件降順」と同義で、内部での選択＋部分ソートとはいえコストが大きいことがあります。
* `np.argsort` は純配列上で高速に動き、**行位置配列→`iloc`** の流れが非常に軽いです。

---

## 3) 追加の細かな最適化ヒント

* **dtype の見直し**

  * `id` を `int32`、`rating` を `float32` に落とせるならメモリ削減（CPU キャッシュ効率↑）。
  * 文字列が多い場合は `pd.StringDtype()`（もしくは pyarrow backend があれば `string[pyarrow]`）でフットプリントを縮小。
* **列アクセスの一貫性**

  * 同じ列を複数回使うときは **一度配列化して再利用**（上記コードのように `to_numpy()` を 1 回だけ呼ぶ）。
* **条件の確定順序**

  * 選択性が高い条件（今回なら `id & 1` よりも `description` 判定の方が効くケースが多い）を先に評価しても、NumPy のブール演算は短絡しないため実行順で速度は大きく変わりません。配列化して一発で作る方が速いです。

---

## 4) 期待効果の目安

* 低リスク版：中間コピー削減と `Series.nlargest` 採用で **10–25% 程度短縮**が見込めることが多いです。
* NumPy 版：データサイズや列数にもよりますが、**さらに数ms〜数十ms** 改善するケースがあります。

> まずは **低リスク版** → 効果が物足りなければ **NumPy 版**、の順でお試しを。
> それでもまだ詰める必要があれば、dtype 最適化や前処理段階でのフィルタ（上流で奇数IDだけ渡す等）をご検討ください。

